In [1]:
import pandas as pd
import json

In [2]:
df1 = pd.read_csv('review1.csv')
df1=df1.drop(['review_id','funny','cool'], axis=1)

In [3]:
df2 = pd.read_csv('review2.csv')
df2=df2.drop(['review_id','funny','cool'], axis=1)

In [4]:
df3 = pd.read_csv('review3.csv')
df3=df3.drop(['review_id','funny','cool'], axis=1)

In [5]:
df4 = pd.read_csv('review4.csv')
df4=df4.drop(['review_id','funny','cool'], axis=1)

In [6]:
df5 = pd.read_csv('review5.csv')
df5=df5.drop(['review_id','funny','cool'], axis=1)

In [7]:
df6 = pd.read_csv('review6.csv')
df6=df6.drop(['review_id','funny','cool'], axis=1)

In [8]:
df7 = pd.read_csv('review7.csv')
df7=df7.drop(['review_id','funny','cool'], axis=1)

In [9]:
df8 = pd.read_csv('review8.csv')
df8=df8.drop(['review_id','funny','cool'], axis=1)

In [10]:
df9 = pd.read_csv('review9.csv')
df9=df9.drop(['review_id','funny','cool'], axis=1)

In [11]:
df10 = pd.read_csv('review10.csv')
df10=df10.drop(['review_id','funny','cool'], axis=1)

In [12]:
df11 = pd.read_csv('review11.csv')
df11=df11.drop(['review_id','funny','cool'], axis=1)

In [13]:
df12 = pd.read_csv('review12.csv')
df12=df12.drop(['review_id','funny','cool'], axis=1)

In [14]:
df13 = pd.read_csv('review13.csv')
df13=df13.drop(['review_id','funny','cool'], axis=1)

In [15]:
df14 = pd.read_csv('review14.csv')
df14=df14.drop(['review_id','funny','cool'], axis=1)

In [16]:
df15 = pd.read_csv('review15.csv')
df15=df15.drop(['review_id','funny','cool'], axis=1)

In [17]:
df16 = pd.read_csv('review16.csv')
df16=df16.drop(['review_id','funny','cool'], axis=1)

In [18]:
df17 = pd.read_csv('review17.csv')
df17=df17.drop(['review_id','funny','cool'], axis=1)

In [19]:
df18 = pd.read_csv('review18.csv')
df18=df18.drop(['review_id','funny','cool'], axis=1)

In [ ]:
#create a dataframe with all the reviews but without the text which is not relevant for the analysis
frames = [df1, df2, df3,df4, df5, df6,df7, df8, df9,df10, df11, df12,df13, df14, df15,df16, df17, df18]
df_r = pd.concat(frames)

In [ ]:
df_r.head(2)

In [ ]:
import numpy as np
import dask.bag as db
import dask.dataframe as dd # substitute of pandas to load the data in streams
import dask.array as da

In [ ]:
df_r.head()

In [ ]:
!pip install nltk

In [ ]:
import nltk
import string
nltk.download("stopwords")

In [ ]:
from nltk.corpus import stopwords
sw_nltk = stopwords.words('english')
print(sw_nltk)

In [ ]:
#n = len(df_r['text'])
df = df_r.sample(10000)
nt = []
for i in range(10000):
  s = str(df['text'].values[i])
  words = [word for word in s.split() if word.lower() not in sw_nltk]
  new_text = " ".join(words)
  new_text = new_text.translate(str.maketrans('', '', string.punctuation))
  nt.append(new_text.lower()) 
df['new_text'] = nt

In [ ]:
df

In [ ]:
!pip install gensim

In [ ]:
import gensim

In [ ]:
from gensim.utils import simple_preprocess
import gensim.corpora as corpora

In [ ]:
def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

In [ ]:
data = df.new_text.values.tolist()
data_words = list(sent_to_words(data))

In [ ]:
id2word = corpora.Dictionary(data_words)

In [ ]:
texts = data_words
corpus = [id2word.doc2bow(text) for text in texts]

In [ ]:
from pprint import pprint
# number of topics
num_topics = 10
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

In [ ]:
df=df.drop(['date','useful','user_id','business_id'],axis=1)   

In [ ]:
for i in range(10):
    s=str(i)
    df[s]=np.zeros(len(df))
for i in range(len(df)):
    l=lda_model.get_document_topics(corpus[i])
    for a in l:
        df[str(a[0])]=a[1]

In [ ]:
review_topics = lda_model.get_document_topics(corpus[100])

In [ ]:
type(review_topics[0][0])

In [ ]:
df.head(9)

In [ ]:
!pip install pyLDAvis

In [ ]:
import pyLDAvis.gensim_models
import pickle 
import pyLDAvis
import os
# Visualize the topics
pyLDAvis.enable_notebook()
LDAvis_data_filepath = os.path.join('./words_in_review2_'+ str(num_topics))
# # this is a bit time consuming - make the if statement True
# # if you want to execute visualization prep yourself
if 1 == 1:
    LDAvis_prepared = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)
# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)
pyLDAvis.save_html(LDAvis_prepared,'./words_in_review2_'+ str(num_topics) +'.html')
LDAvis_prepared

In [ ]:
import sklearn 
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error

In [ ]:
train, test = train_test_split(df, test_size=0.2)

In [ ]:
train.head()

In [ ]:
x_train, y_train = train.drop(columns = ['text', 'new_text', 'stars']), train['stars']

In [ ]:
parameters = {'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}


In [ ]:
rf_model = RandomForestRegressor()

#we perform a grid search to find the best parameters in the param dictionary created before 
clf = RandomizedSearchCV(estimator= rf_model, param_distributions = parameters, n_iter = 20, scoring = 'neg_mean_squared_error', verbose=2, n_jobs=-1)
#we fit the model with the best parameters
clf.fit(x_train, y_train)

In [ ]:
x_test, y_test = test.drop(columns = ['text', 'new_text', 'stars']), test['stars']

In [ ]:
y_pred = clf.predict(x_test)

In [ ]:
MSE = np.mean((y_pred-y_test)**2)